In [1]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 65 Apr 26 19:34 kaggle.json


In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
# List available datasets.
!kaggle datasets list

ref                                                            title                                                size  lastUpdated          downloadCount  
-------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
allen-institute-for-ai/CORD-19-research-challenge              COVID-19 Open Research Dataset Challenge (CORD-19)    2GB  2020-04-25 00:03:33          57337  
roche-data-science-coalition/uncover                           UNCOVER COVID-19 Challenge                          123MB  2020-04-08 22:04:30           5389  
praveengovi/coronahack-chest-xraydataset                       CoronaHack -Chest X-Ray-Dataset                       1GB  2020-03-20 01:26:40           1434  
katerpillar/meteonet                                           MeteoNet North-West France                           11GB  2020-04-16 08:35:45            113  
andrewmvd/covid19-ct-scans                    

In [5]:
import datetime, os, subprocess
def update_dataset():
    DOWNLOADPATH = '/content/'
    tday = str(datetime.date.today())
    data = "updated_on_{}".format(tday)
    toCreate = DOWNLOADPATH + data
    unzip_the_files = "unzip {} -d {}".format(DOWNLOADPATH + "novel-corona-virus-2019-dataset.zip", toCreate)
    print(unzip_the_files)
    os.makedirs(toCreate)
    !kaggle datasets download sudalairajkumar/novel-corona-virus-2019-dataset
    error = os.system(unzip_the_files)
    if not error:
        print("Updated the dataset!")

update_dataset()

unzip /content/novel-corona-virus-2019-dataset.zip -d /content/updated_on_2020-04-26
  0% 0.00/833k [00:00<?, ?B/s]
100% 833k/833k [00:00<00:00, 56.4MB/s]
Updated the dataset!


In [6]:
!unzip *.zip

Archive:  novel-corona-virus-2019-dataset.zip
  inflating: COVID19_line_list_data.csv  
  inflating: COVID19_open_line_list.csv  
  inflating: covid_19_data.csv       
  inflating: time_series_covid_19_confirmed.csv  
  inflating: time_series_covid_19_confirmed_US.csv  
  inflating: time_series_covid_19_deaths.csv  
  inflating: time_series_covid_19_deaths_US.csv  
  inflating: time_series_covid_19_recovered.csv  


In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.layers import Dense
from keras.models import Sequential

In [220]:
import pandas as pd
import numpy as np
data = pd.read_csv('/content/COVID19_line_list_data.csv', sep=',')
data.describe()

,id,case_in_country,Unnamed: 3,age,If_onset_approximated,visiting Wuhan,from Wuhan,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
count,1085.000000,888.000000,0.0,843.000000,560.000000,1085.000000,1081.000000,0.0,0.0,0.0,0.0,0.0,0.0
mean,543.000000,48.841216,NaN,49.483689,0.042857,0.176959,0.144311,NaN,NaN,NaN,NaN,NaN,NaN
std,313.356825,78.853528,NaN,18.255334,0.202716,0.381810,0.351567,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,1.000000,NaN,0.250000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,272.000000,11.000000,NaN,35.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,543.000000,28.000000,NaN,51.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,814.000000,67.250000,NaN,64.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
max,1085.000000,1443.000000,NaN,96.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
df_str = data[['summary', 'gender', 'symptom']]

In [0]:
df_int = data[['reporting date', 'location', 'country', 'age',
             'visiting Wuhan', 'from Wuhan', 'death', 'recovered']]

In [224]:
df_str.head()

,summary,gender,symptom
0,First confirmed imported COVID-19 pneumonia pa...,male,NaN
1,First confirmed imported COVID-19 pneumonia pa...,female,NaN
2,First confirmed imported cases in Zhejiang: pa...,male,NaN
3,new confirmed imported COVID-19 pneumonia in T...,female,NaN
4,new confirmed imported COVID-19 pneumonia in T...,male,NaN


In [225]:
df_int.head()

,reporting date,location,country,age,visiting Wuhan,from Wuhan,death,recovered
0,1/20/2020,"Shenzhen, Guangdong",China,66.0,1,0.0,0,0
1,1/20/2020,Shanghai,China,56.0,0,1.0,0,0
2,1/21/2020,Zhejiang,China,46.0,0,1.0,0,0
3,1/21/2020,Tianjin,China,60.0,1,0.0,0,0
4,1/21/2020,Tianjin,China,58.0,0,0.0,0,0


In [0]:
df_str = df_str.fillna(' ')

In [0]:
df_int = df_int.fillna(0)

In [0]:
def map_dates_to_ints(date_val):
  date_val = str(date_val)
  if date_val != '0':
    split_date = date_val.split('/')
    initial = int(split_date[0])
    final = int(split_date[1])
    result = int((initial * final ** 2) ** 0.5)
  else:
    result = 0
  return (result)

In [0]:
df_int['reporting date'] = df_int['reporting date'].apply(lambda x: map_dates_to_ints(x))

In [231]:
import nltk
nltk.download('stopwords')
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [0]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            pos = pos_tag([i.strip()])
            word = lemmatizer.lemmatize(i.strip(),get_simple_pos(pos[0][1]))
            final_text.append(word.lower())
    return " ".join(final_text) 

In [234]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
df_str.summary = df_str.summary.apply(lemmatize_words)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
data_combined = pd.concat([df_int, df_str], axis=1, sort=False)

In [236]:
data_combined.head()

,reporting date,location,country,age,visiting Wuhan,from Wuhan,death,recovered,summary,gender,symptom
0,20,"Shenzhen, Guangdong",China,66.0,1,0.0,0,0,first confirm import covid-19 pneumonia patien...,male,
1,20,Shanghai,China,56.0,0,1.0,0,0,first confirm import covid-19 pneumonia patien...,female,
2,21,Zhejiang,China,46.0,0,1.0,0,0,first confirm import case zhejiang: patient ma...,male,
3,21,Tianjin,China,60.0,1,0.0,0,0,new confirm import covid-19 pneumonia tianjin:...,female,
4,21,Tianjin,China,58.0,0,0.0,0,0,new confirm import covid-19 pneumonia tianjin:...,male,


In [0]:
#import the necessary module
from sklearn import preprocessing
import re
# create the Labelencoder object
le = preprocessing.LabelEncoder()
#convert the categorical columns into numeric
data_combined['summary'] = le.fit_transform(data_combined['summary'])
data_combined['location'] = le.fit_transform(data_combined['location'])
data_combined['country'] = le.fit_transform(data_combined['country'])
data_combined['gender'] = le.fit_transform(data_combined['gender'])
data_combined['symptom'] = le.fit_transform(data_combined['symptom'])

In [114]:
data_combined.head()

,reporting date,summary,location,country,gender,age,symptom,visiting Wuhan,from Wuhan,death,recovered
0,20,67,116,8,2,66,0,1,0,0,0
1,20,65,113,8,1,56,0,0,1,0,0
2,21,51,154,8,2,46,0,0,1,0,0
3,21,938,129,8,1,60,0,1,0,0,0
4,21,939,129,8,2,58,0,0,0,0,0


In [0]:
X = data_combined.filter(['reporting date', 'summary', 'location', 'country', 'gender', 'age', 'symptom',
             'visiting Wuhan', 'from Wuhan'], axis=1)
y1 = data_combined['death']
y2 = data_combined['recovered']

In [250]:
y1.shape

(1085,)

In [240]:
X.head()

,reporting date,summary,location,country,gender,age,symptom,visiting Wuhan,from Wuhan
0,20,67,116,8,2,66.0,0,1,0.0
1,20,65,113,8,1,56.0,0,0,1.0
2,21,51,154,8,2,46.0,0,0,1.0
3,21,938,129,8,1,60.0,0,1,0.0
4,21,939,129,8,2,58.0,0,0,0.0


In [0]:
x_train, x_test, y1_train, y1_test = train_test_split(X, y1 , test_size = 0.2 , random_state = 0)

In [0]:
x_train, x_test, y2_train, y2_test = train_test_split(X, y2 , test_size = 0.2 , random_state = 0)

In [288]:
from sklearn.svm import SVC
# Try with Polynomial kernel SVC
polySVC = SVC(kernel='poly', degree=8)
clf = polySVC.fit(x_train, y1_train)
pred = clf.predict(x_test)
print("PolySVC accuracy : ",accuracy_score(y1_test, pred, normalize = True))

PolySVC accuracy :  0.9539170506912442


In [289]:
from sklearn.svm import SVC
# Try with Polynomial kernel SVC
polySVC = SVC(kernel='poly', degree=8)
clf = polySVC.fit(x_train, y2_train)
pred = clf.predict(x_test)
print("PolySVC accuracy : ",accuracy_score(y2_test, pred, normalize = True))

PolySVC accuracy :  0.8663594470046083


In [272]:
#import necessary modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
#create object of the lassifier
neigh = KNeighborsClassifier(n_neighbors=3)
#Train the algorithm
neigh.fit(x_train, y1_train)
# predict the response
pred = neigh.predict(x_test)
# evaluate accuracy
print ("KNeighbors accuracy score : ",accuracy_score(y1_test, pred))

KNeighbors accuracy score :  0.9585253456221198


In [291]:
#import necessary modules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
#create object of the lassifier
neigh = KNeighborsClassifier(n_neighbors=3)
#Train the algorithm
neigh.fit(x_train, y2_train)
# predict the response
pred = neigh.predict(x_test)
# evaluate accuracy
print ("KNeighbors accuracy score : ",accuracy_score(y2_test, pred))

KNeighbors accuracy score :  0.8617511520737328


In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
def create_baseline():
  model = Sequential()
  model.add(Dense(50, input_dim=9, activation='relu'))
  model.add(Dense(units = 40 , activation = 'relu'))
  model.add(Dense(units = 35 , activation = 'relu'))
  model.add(Dense(units = 25 , activation = 'relu'))
  model.add(Dense(units = 10 , activation = 'relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [0]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=20, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, x_train, y1_train, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


Baseline: 81.46% (22.38%)
